# Data collection

To collect the relevant wikipedia pages for our project we specify the dataclass `Node`. This is based on the use of the open-source software [PetScan](https://petscan.wmflabs.org/) that based on a list of wikipedia-categories yields the corresponding page-names. We furthermore specify the depth of our PetScan-query, which is a measure of how deep we want our categories to be. As the list of pages grows exponentially with the levels of depth we set the parameter to 1. 

In [1]:
import requests
import json
import wikipedia
import pandas as pd
from tqdm.notebook import tqdm
from dataclasses import dataclass
from typing import List
from itertools import chain
from collections import Counter

In [12]:
# We define the dataclass Node that contains: name, parent, depth, edges, text and categories of a page
@dataclass(frozen=False)
class Node:
    name:str
    parent:str
    depth:int=0
    edges:List[tuple]=None
    text:str=None
    categories:List[str]=None    
    

def collect_nodes(parents:list,depth:int=0)->List[Node]:
    
    """
    Based on a category(parents) and the depth this function collects all of the 
    corresponding wikipedia pages and returns a list of element of classtype Node.
    """
    
    nodes = list()
    base_url = 'https://petscan.wmflabs.org/?ns%5B0%5D=1&'
    params = {'project':'wikipedia',
              'language':'en',
              'format':'json',
              'interface_language':'en',
              'depth':str(depth),
              'doit':''}
    
    # Loop through all the categories of the parent-category
    for cat in tqdm(parents):
        
        params['categories'] = cat  # Add new entry to the params dict
        resp = requests.get(url=base_url, params=params).json() #  Get pages based on the params
        
        # Due to DisambiguationError (some page query are disambiguats) we have to use try statements
        # to extract the page elements
        try: 
            for node in tqdm(resp['*'][0]['a']['*'], leave=False):
                try:
                    article = wikipedia.page(node['title'].replace('_',' '))
                    text = article.content
                    edges = article.links
                    categories = article.categories
                    #Append the elements to nodes
                    nodes.append(Node(name=node['title'],
                                    parent=cat,
                                    depth=depth,
                                    edges=edges,
                                    text=text,
                                    categories=categories))
                #except AttributeError or DisambiguationError:
#                    nodes.append(Node(name=node['title'],
#                                    parent=cat,
#                                    depth=depth))
                except:
                    print(f'Node {node.name} could not be collected...')
            
        except KeyError:
            print(f'Category {cat} could not be collected...')
    
    return nodes

In [15]:
# We now make our actual query be specying, parent-categories and depth
parent_categories = ['political_science', 'economics', 'sociology', 'anthropology', 'psychology']
depths = [1]
nodes = []
for d in tqdm(depths):
    nodes += collect_nodes(parent_categories, d)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/892 [00:00<?, ?it/s]

Node {'id': 21754, 'len': 22738, 'metadata': {'wikidata': 'Q6266'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q6266', 'title': 'Nation', 'touched': '20211111144259'} could not be collected...
Node {'id': 408891, 'len': 4910, 'metadata': {'wikidata': 'Q5172479'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q5172479', 'title': 'Corporate_nationalism', 'touched': '20210919002625'} could not be collected...
Node {'id': 454402, 'len': 25655, 'metadata': {'wikidata': 'Q617473'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q617473', 'title': 'Likert_scale', 'touched': '20211016074109'} could not be collected...
Node {'id': 467982, 'len': 12654, 'metadata': {'wikidata': 'Q1761743'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q1761743', 'title': 'Red_tape', 'touched': '20211107200515'} could not be collected...
Node {'id': 503474, 'len': 25323, 'metadata': {'wikidata': 'Q1752412'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q1752412', 'title': 'Nation-building', 'to

  0%|          | 0/987 [00:00<?, ?it/s]

...

  0%|          | 0/1843 [00:00<?, ?it/s]

Node {'id': 16743, 'len': 192133, 'metadata': {'wikidata': 'Q9061'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q9061', 'title': 'Karl_Marx', 'touched': '20211118222647'} could not be collected...
Node {'id': 17547, 'len': 69654, 'metadata': {'wikidata': 'Q316'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q316', 'title': 'Love', 'touched': '20211105005337'} could not be collected...
Node {'id': 30299, 'len': 120720, 'metadata': {'wikidata': 'Q194100'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q194100', 'title': 'Transhumanism', 'touched': '20211117175004'} could not be collected...
Node {'id': 50024, 'len': 31673, 'metadata': {'wikidata': 'Q6498477'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q6498477', 'title': 'Lust', 'touched': '20211108212706'} could not be collected...
Node {'id': 154147, 'len': 37499, 'metadata': {'wikidata': 'Q1210705'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q1210705', 'title': 'Limerence', 'touched': '20211017073138'} could

Node {'id': 26266653, 'len': 9629, 'metadata': {'wikidata': 'Q192242'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q192242', 'title': 'Frustration', 'touched': '20210928184310'} could not be collected...
Node {'id': 27431805, 'len': 8831, 'metadata': {'wikidata': 'Q7495646'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q7495646', 'title': 'Shetani', 'touched': '20210523204424'} could not be collected...
Node {'id': 29331449, 'len': 18281, 'metadata': {'wikidata': 'Q14623169'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q14623169', 'title': 'Nations_and_IQ', 'touched': '20211014025022'} could not be collected...
Node {'id': 34104355, 'len': 50917, 'metadata': {'wikidata': 'Q7456400'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q7456400', 'title': 'Subjective_well-being', 'touched': '20211108130206'} could not be collected...
Node {'id': 36196531, 'len': 5387, 'metadata': {'wikidata': 'Q5508847'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q5508847', 'title':

  0%|          | 0/1374 [00:00<?, ?it/s]

Node {'id': 10332, 'len': 69285, 'metadata': {'wikidata': 'Q59157'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q59157', 'title': 'Educational_psychology', 'touched': '20211114002059'} could not be collected...
Node {'id': 28297, 'len': 94037, 'metadata': {'wikidata': 'Q9165'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q9165', 'title': 'Soul', 'touched': '20211118164536'} could not be collected...
Node {'id': 36770, 'len': 9557, 'metadata': {'wikidata': 'Q324925'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q324925', 'title': 'Tony_Buzan', 'touched': '20210623075754'} could not be collected...
Node {'id': 102883, 'len': 96377, 'metadata': {'wikidata': 'Q34394'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q34394', 'title': 'Belief', 'touched': '20211021183808'} could not be collected...
Node {'id': 238212, 'len': 62446, 'metadata': {'wikidata': 'Q1987782'}, 'n': 'page', 'namespace': 0, 'nstext': '', 'q': 'Q1987782', 'title': 'Abnormal_psychology', 'touched': '20

Following the collection of pages we gather them in a dataframe and edgelist for future use. To reduce the size of edgelist we alreay now remove edges that points to pages we have not collected. This means that we only keep edges that link to other pages in one of the five categories. 

In [129]:
# Based on our nodes we can now create and save our df for future use
def create_df(nodes = nodes):
    return pd.DataFrame({"name": [node.name for node in nodes],
                         "parent": [node.parent for node in nodes],
                         "depth": [node.depth for node in nodes],
                         "edges": [node.edges for node in nodes],
                         "text": [node.text for node in nodes],
                         "categories": [node.categories for node in nodes]})

df = create_df()
df.to_pickle("df.obj")

In [130]:
# Based on our nodes we can now create a edgelist and save our ot for future use
def create_edgelist(nodes = nodes):
    nodelist = [node.name for node in nodes]
    edgelist = [[(nodes[i].name, edge) for edge in nodes[i].edges if edge in nodelist]
                for i in tqdm(range(len(nodes)))]
    return list(chain.from_iterable(edgelist))

edgelist = create_edgelist()
with open('edgelist.obj', 'wb') as f:
    pickle.dump(edgelist, f)